In [1]:
import geemap
import ee
geemap.ee_initialize()


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=bqCrI1vgDrvO7hMz22TfKiT5yGUwHm0oo-RH2i_W_DU&tc=bRt4X6ndXYPm9l5dsGSB0_Nd_49dKKXDD8rprTrT-GU&cc=LnSTy7QhecOKVUU09soWAfVfY65W4o4guxKvvh3FWSc

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AeaYSHCsKUuBD2Ki4KfVRPZVfR3hahbJNiUxwBQR7ce9Q50KcSh4YWcxt54

Successfully saved authorization token.


**Question 1**

In [29]:
from geemap.datasets import DATA
Map = geemap.Map()
fc =ee.FeatureCollection(DATA.TIGER_2018_Counties)
county= fc.filter(ee.Filter.eq('NAME', 'Knox'))
(geemap.ee_to_df(county)) #Table format
#print(str(len(geemap.ee_to_df(county))) + " counties named Knox")

,ALAND,AWATER,CBSAFP,CLASSFP,COUNTYFP,COUNTYNS,CSAFP,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,LSAD,METDIVFP,MTFCC,NAME,NAMELSAD,STATEFP
0,1855569287,8837382,23660,H1,095,00424249,,A,17095,+40.9309446,-090.2137921,06,,G4020,Knox,Knox County,17
1,1336511196,20758105,47180,H1,083,00450366,,A,18083,+38.6883910,-087.4203544,06,,G4020,Knox,Knox County,18
2,1000523138,3743713,30940,H1,121,00516907,,A,21121,+36.8874755,-083.8556297,06,,G4020,Knox,Knox County,21
3,945684684,2017116243,,H1,013,00581292,,A,23013,+44.0420446,-069.0385145,06,,G4020,Knox,Knox County,23
4,1305376626,7307094,,H1,103,00758506,,A,29103,+40.1368559,-092.1468345,06,,G4020,Knox,Knox County,29
5,2870856403,81009708,,H1,107,00835875,,A,31107,+42.6344045,-097.8913492,06,,G4020,Knox,Knox County,31
6,1361052752,10720134,34540,H1,083,01074054,198,A,39083,+40.4036198,-082.4223929,06,,G4020,Knox,Knox County,39
7,1316572232,45558452,28940,H1,093,01639761,314,A,47093,+35.9927265,-083.9377209,06,,G4020,Knox,Knox County,47
8,2203109026,12577260,,H1,275,01383923,,A,48275,+33.6118897,-099.7303563,06,,G4020,Knox,Knox County,48


**Question 2**

In [30]:
Map=geemap.Map()
data =ee.FeatureCollection(DATA.TIGER_2018_Counties)
subset= fc.filter(ee.Filter.eq('GEOID', '47093'))
vis = {
    'color': 'FF000000',
    'lineType': 'solid',
}
Map.centerObject(subset, zoom=10)
Map.addLayer(subset,vis,'Knoxville')
Map.add_text('Knoxville by Javier Martinez', fontsize=20, position= 'bottomright')
Map

Map(center=[35.993196068178555, -83.93723059189078], controls=(WidgetControl(options=['position', 'transparent…

**Question 3**

In [53]:
Map=geemap.Map()
region2 =ee.FeatureCollection(DATA.TIGER_2018_Counties).filter(ee.Filter.eq('GEOID', '47093'))
vis = {
    'color': '00000000',
    'lineType': 'solid',
}
Map.centerObject(region2, zoom=10)
collection = (
    ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
    .filterBounds(region2)
    .filter(ee.Filter.date('2022-09-01', '2023-12-31'))
)

# Applies scaling factors (function from the Data Catalog)
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )
dataset = collection.map(apply_scale_factors)
landsat=dataset.median()
visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}
clip=landsat.clipToCollection(region2)
Map.addLayer(clip, visualization, 'Landsat')
Map.addLayer(region2,vis,'Knoxville')
Map.add_text('Landsat Knoxville by Javier', fontsize=20, position= 'topright')
Map

Map(center=[35.993196068178555, -83.93723059189078], controls=(WidgetControl(options=['position', 'transparent…

**Question 4**

In [54]:
Map=geemap.Map()
region2 =ee.FeatureCollection(DATA.TIGER_2018_Counties).filter(ee.Filter.eq('GEOID', '47093'))
vis = {
    'color': '00000000',
    'lineType': 'solid',
}
Map.centerObject(region2, zoom=10)
collection = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterBounds(region2)
    .filter(ee.Filter.date('2023-09-01', '2023-12-31'))
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than', 10)
)
sent=collection.median()
visualization = {
    "min": 0.0,
    "max": 4000,
    "bands": ['B3', 'B4', 'B2'],
}
clip=sent.clipToCollection(region2)
Map.addLayer(clip, visualization, 'Sentinel2')
Map.addLayer(region2,vis,'Knoxville')
Map.add_text('Senmtinel 2 Knoxville by Javier', fontsize=20, position= 'topright')
Map

Map(center=[35.993196068178555, -83.93723059189078], controls=(WidgetControl(options=['position', 'transparent…

**Question 5**

In [56]:
Map=geemap.Map()
region2 =ee.FeatureCollection(DATA.TIGER_2018_Counties).filter(ee.Filter.eq('GEOID', '47093'))
vis = {
    'color': '00000000',
    'lineType': 'solid',
}
Map.centerObject(region2, zoom=10)
collection = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterBounds(region2)
    .filter(ee.Filter.date('2020-09-01', '2022-12-12'))
)
naip=collection.median()
visualization = {
    'min': 0.0,
    'max': 255,
    'bands': ['R', 'G', 'B'],
}
clip=naip.clipToCollection(region2)
Map.addLayer(clip, visualization, 'Agriculture NAIP')
Map.addLayer(region2,vis,'Knoxville')
Map.add_text('Agricultural Knoxville by Javier', fontsize=20, position= 'topleft')
Map

Map(center=[35.993196068178555, -83.93723059189078], controls=(WidgetControl(options=['position', 'transparent…